In [1]:
import numpy as np 
import pandas as pd 
import tensorflow as tf 
import keras
from keras.layers import Dense, LSTM, SimpleRNN, GRU, Input
from keras.models import Model, Sequential  

df_train = pd.read_csv("./storage/Kaggle_M5_Luke/sales_train_validation.csv")
df_sell = pd.read_csv("./storage/Kaggle_M5_Luke/sell_prices.csv")
df_calendar = pd.read_csv("./storage/Kaggle_M5_Luke/calendar.csv")
df_ss = pd.read_csv("./storage/Kaggle_M5_Luke/sample_submission_m5.csv")


Using TensorFlow backend.


In [2]:
days = range(1,1913+1) 
time_series_cols = [] 
for i in days: 
    time_series_cols.append('d_'+str(i)) 
time_series_cols[:10]

['d_1', 'd_2', 'd_3', 'd_4', 'd_5', 'd_6', 'd_7', 'd_8', 'd_9', 'd_10']

In [3]:
time_series_data = df_train[time_series_cols] 
time_series_data

,d_1,d_2,d_3,d_4,d_5,d_6,d_7,d_8,d_9,d_10,...,d_1904,d_1905,d_1906,d_1907,d_1908,d_1909,d_1910,d_1911,d_1912,d_1913
0,0,0,0,0,0,0,0,0,0,0,...,1,3,0,1,1,1,3,0,1,1
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,2,1,2,1,1,1,0,1,1,1
3,0,0,0,0,0,0,0,0,0,0,...,1,0,5,4,1,0,1,3,7,2
4,0,0,0,0,0,0,0,0,0,0,...,2,1,1,0,1,1,2,2,2,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0,0,2,2,0,3,1,4,1,0,...,2,0,0,0,0,0,1,0,0,1
30486,0,0,0,0,0,5,0,1,1,3,...,0,0,0,0,0,0,0,0,1,0
30487,0,6,0,2,2,4,1,8,5,2,...,2,1,0,2,0,1,0,0,1,0
30488,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,1,0,3,1,3


In [4]:
time_series_data = np.asarray(time_series_data) 
print(time_series_data.shape)

(30490, 1913)


In [6]:
def build_model(time_steps,features): 
    model = Sequential() 
    model.add(LSTM(8,input_shape=(time_steps,features),activation='relu',return_sequences=True)) 
    model.add(Dense(features)) 
    return model 
#model = build_model(7,30490)
#model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 7, 8)              975968    
_________________________________________________________________
dense_1 (Dense)              (None, 7, 30490)          274410    
Total params: 1,250,378
Trainable params: 1,250,378
Non-trainable params: 0
_________________________________________________________________


In [12]:
# making predictions for the 28 days 
# x_train: d_i to d_(i+6), y_train: d_(i+1) to d_(i+7), if time_steps = 7 
# leave last 7 day window for validation 
def split_xy1(dataset,time_steps): 
    x,y = list(), list() 
    for i in range(0,dataset.shape[1]-time_steps): 
        x.append(dataset[:,i:i+time_steps]) 
        y.append(dataset[:,i+1:i+time_steps+1]) 
    return np.array(x),np.array(y)   

# the important thing about this model, is that for predictions[0], we need to extract the final timestep from the 7 timesteps we have for prediction. 
predictions = [] 
for i in range(1,29): 
    print("day {}".format(i)) 
    dataset = time_series_data[:,::i] 
    x_train, y_train = split_xy1(dataset,7) 
    x_pred = dataset[:,dataset.shape[1]-7:]
    x_train = x_train.reshape(x_train.shape[0],7,30490) 
    y_train = y_train.reshape(y_train.shape[0],7,30490) 
    x_pred = x_pred.reshape(1,7,30490) 
    model = build_model(7,30490)
    model.compile(loss='mse',optimizer='adam',metrics=['mse']) 
    model.fit(x_train,y_train,epochs=100) 
    pred = model.predict(x_pred) 
    predictions.append(pred) 


day 1
Epoch 1/100
1906/1906 [==============================] - 6s 3ms/step - loss: 16.1948 - mse: 16.1948
Epoch 2/100
1906/1906 [==============================] - 6s 3ms/step - loss: 16.0706 - mse: 16.0706
Epoch 3/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.9557 - mse: 15.9557
Epoch 4/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.8497 - mse: 15.8497
Epoch 5/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.7516 - mse: 15.7516
Epoch 6/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.6609 - mse: 15.6609
Epoch 7/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.5769 - mse: 15.5769
Epoch 8/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.4990 - mse: 15.4990
Epoch 9/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.4266 - mse: 15.4266
Epoch 10/100
1906/1906 [==============================] - 6s 3ms/step - loss: 15.3594 - mse: 1

In [27]:
print("checking out the shape of prediction")
print("test 1")
print(predictions[0]) 
print("test 2") 
print(predictions[0][0]) # array is not flattened  
print("test3")  
print(predictions[0][0][0])

checking out the shape of prediction
test 1
[[[3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
  [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
  [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
  ...
  [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
  [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
  [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]]]
test 2
[[3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
 [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
 [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
 ...
 [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
 [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]
 [3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]]
test3
[3.0652962 3.0518794 3.1949353 ... 3.0707428 3.1151094 3.1163583]


In [28]:
pred_extracted = [] # extract the last column for each obtained prediction 
for i in range(28): 
    pred_extracted.append(predictions[i][0][6])

In [33]:
cols = [] 
for i in range(1,29): 
    cols.append("F"+str(i)) 
df = pd.DataFrame(pred_extracted)   
df = df.T # transpose 
df.columns = cols # rename column 
df 

,F1,F2,F3,F4,F5,F6,F7,F8,F9,F10,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,3.065296,2.310839,2.519437,2.167134,1.553728,0.919739,1.581925,3.519936,1.778415,2.257769,...,2.758889,4.834609,0.294185,2.188495,0.293039,0.291588,2.395867,0.292583,2.458565,0.197706
1,3.051879,2.305327,2.559673,2.308092,1.522447,0.918921,1.431946,3.465383,1.703900,2.249476,...,2.651120,4.683928,0.294721,2.138725,0.292321,0.291391,2.401983,0.289991,2.394089,0.197675
2,3.194935,2.345295,2.949124,2.374190,1.795300,0.922335,1.685668,3.581693,1.823510,2.427212,...,2.913798,4.877257,0.294243,2.333577,0.293345,0.291310,2.591061,0.292168,2.633536,0.197817
3,3.195315,2.344229,2.930589,2.537106,1.808359,0.922086,1.647720,3.792625,1.867725,2.317297,...,2.967013,4.917827,0.294943,2.322122,0.292498,0.292864,2.751157,0.292733,2.538388,0.197802
4,0.676321,0.663352,1.128408,1.331771,0.731414,0.566104,1.877141,0.708870,0.743134,0.771997,...,1.188148,1.268930,0.268933,0.683082,0.260048,0.257076,1.018465,0.249283,0.851990,0.187349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
30485,0.744723,0.748629,0.726185,0.697736,0.671602,0.646473,0.754335,0.873214,0.580832,0.452599,...,0.911161,1.116568,0.271318,0.882870,0.268111,0.269107,0.635701,0.267073,0.775352,0.189290
30486,0.742392,0.748308,0.365864,0.818646,0.717397,0.646024,0.844109,0.850631,0.739699,0.482372,...,0.856885,1.140350,0.269497,0.798038,0.265315,0.270850,0.574009,0.263502,0.786455,0.188636
30487,3.070743,2.314696,2.212970,2.238082,2.929354,0.920788,1.013309,3.558037,4.206920,1.914698,...,5.293853,4.650221,0.294400,4.428310,0.293233,0.291228,4.712502,0.291492,5.193668,0.197875
30488,3.115109,2.328309,2.183475,2.039086,3.016931,0.921223,0.927830,3.742261,4.188385,1.898798,...,5.292744,4.826906,0.294178,4.555754,0.292541,0.293091,4.942726,0.291892,5.261531,0.197896


In [34]:
validation_ids = df_train['id'].values
evaluation_ids = [i.replace('validation', 'evaluation') for i in validation_ids]
ids = np.concatenate([validation_ids, evaluation_ids])       
ids

array(['HOBBIES_1_001_CA_1_validation', 'HOBBIES_1_002_CA_1_validation',
       'HOBBIES_1_003_CA_1_validation', ...,
       'FOODS_3_825_WI_3_evaluation', 'FOODS_3_826_WI_3_evaluation',
       'FOODS_3_827_WI_3_evaluation'], dtype=object)

In [35]:
sub_df = pd.DataFrame(ids, columns=['id']) 
df = pd.concat([df]*2).reset_index(drop=True) 
sub_df = pd.concat([sub_df,df],axis=1) 
sub_df.to_csv('LSTM_base1.csv',index=False) 

In [36]:
sub_df

,id,F1,F2,F3,F4,F5,F6,F7,F8,F9,...,F19,F20,F21,F22,F23,F24,F25,F26,F27,F28
0,HOBBIES_1_001_CA_1_validation,3.065296,2.310839,2.519437,2.167134,1.553728,0.919739,1.581925,3.519936,1.778415,...,2.758889,4.834609,0.294185,2.188495,0.293039,0.291588,2.395867,0.292583,2.458565,0.197706
1,HOBBIES_1_002_CA_1_validation,3.051879,2.305327,2.559673,2.308092,1.522447,0.918921,1.431946,3.465383,1.703900,...,2.651120,4.683928,0.294721,2.138725,0.292321,0.291391,2.401983,0.289991,2.394089,0.197675
2,HOBBIES_1_003_CA_1_validation,3.194935,2.345295,2.949124,2.374190,1.795300,0.922335,1.685668,3.581693,1.823510,...,2.913798,4.877257,0.294243,2.333577,0.293345,0.291310,2.591061,0.292168,2.633536,0.197817
3,HOBBIES_1_004_CA_1_validation,3.195315,2.344229,2.930589,2.537106,1.808359,0.922086,1.647720,3.792625,1.867725,...,2.967013,4.917827,0.294943,2.322122,0.292498,0.292864,2.751157,0.292733,2.538388,0.197802
4,HOBBIES_1_005_CA_1_validation,0.676321,0.663352,1.128408,1.331771,0.731414,0.566104,1.877141,0.708870,0.743134,...,1.188148,1.268930,0.268933,0.683082,0.260048,0.257076,1.018465,0.249283,0.851990,0.187349
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
60975,FOODS_3_823_WI_3_evaluation,0.744723,0.748629,0.726185,0.697736,0.671602,0.646473,0.754335,0.873214,0.580832,...,0.911161,1.116568,0.271318,0.882870,0.268111,0.269107,0.635701,0.267073,0.775352,0.189290
60976,FOODS_3_824_WI_3_evaluation,0.742392,0.748308,0.365864,0.818646,0.717397,0.646024,0.844109,0.850631,0.739699,...,0.856885,1.140350,0.269497,0.798038,0.265315,0.270850,0.574009,0.263502,0.786455,0.188636
60977,FOODS_3_825_WI_3_evaluation,3.070743,2.314696,2.212970,2.238082,2.929354,0.920788,1.013309,3.558037,4.206920,...,5.293853,4.650221,0.294400,4.428310,0.293233,0.291228,4.712502,0.291492,5.193668,0.197875
60978,FOODS_3_826_WI_3_evaluation,3.115109,2.328309,2.183475,2.039086,3.016931,0.921223,0.927830,3.742261,4.188385,...,5.292744,4.826906,0.294178,4.555754,0.292541,0.293091,4.942726,0.291892,5.261531,0.197896


In [8]:
#model.compile(loss='mse',optimizer='adam',metrics=['mse'])
#model_checkpoint = keras.callbacks.ModelCheckpoint('./storage/Kaggle_M5_Luke/model_base1/model{epoch:02d}.h5',period=50)
#model.fit(x_train,y_train,epochs=1000,callbacks=[model_checkpoint],validation_data=(x_val,y_val))  

